In [1]:
import pandas as pd
import numpy as np
import statistics
def genXDayAverage(df, days, tag):
    toReturn = np.array([0] * df.shape[0])
    toReturn = pd.Series(toReturn, dtype = 'float64')
    for x in range(days):
        shiftedSeries = df[tag].shift(x)
        toReturn = toReturn.add(shiftedSeries)
    return toReturn.div(days)

def stdDevLastXDays(df, days,tag):
    toReturn = []
    pastXDayList = [np.nan] * days
    for _,item in df[tag].items():
        pastXDayList.append(item)
        pastXDayList.pop(0)
        try:
            toReturn.append(statistics.stdev(pastXDayList))
        except:
            toReturn.append(np.nan)
    toReturn = pd.Series(toReturn)
    return toReturn

In [2]:
import json
import requests
import pandas as pd
import numpy as np

apiKey = ''
with open('config/fredKeys.json', 'r') as f:
    data = json.load(f)
    apiKey = data['APIKEY']
fileRoot = 'fredData/'
urlRoot = 'https://api.stlouisfed.org/fred/series/observations?'
seriesOfInterest = ['DTWEXAFEGS','DPRIME', 'TOTCI', 'UNRATE', 'CONSUMER','BUSLOANS','CCLACBW027SBOG']
dateRange = np.arange('2010-01-01', '2020-05-26', dtype='datetime64[D]')
dateRange = {'Date': dateRange}
dateDF = pd.DataFrame(data = dateRange)
seriesDFList= []
for series in seriesOfInterest:
    url = urlRoot + 'series_id='+ series + '&api_key='+ apiKey + '&file_type=json'
    resp = requests.get(url)
    obs = resp.json()['observations']
    dateList = [x['date'] for x in obs]
    valList = [x['value'] for x in obs]
    valueLabel = series + 'Value'
    newDF = {'Date': dateList, valueLabel: valList}
    newDF = pd.DataFrame(data = newDF)
    newDF['Date'] = pd.to_datetime(newDF['Date'])
    
    newDF.replace(['.'], np.nan, inplace = True)
    newDF.fillna(method = 'ffill', inplace = True)
    newDF[valueLabel] = newDF[valueLabel].astype('float64')
    
    newDF[series + 'lastChangeP'] = (newDF[valueLabel] - newDF[valueLabel].shift(1)).div(newDF[valueLabel].shift(1))
    fiveTickAverage = genXDayAverage(newDF, 5, valueLabel)
    tenTickAverage = genXDayAverage(newDF, 10, valueLabel)
    newDF[series + 'fiveVsTenTickAverage'] = (fiveTickAverage - tenTickAverage).div(tenTickAverage)
    
    ticks = 10
    Std = stdDevLastXDays(newDF, ticks,valueLabel)
    Average = genXDayAverage(newDF, ticks, valueLabel)
    upperBand = ((Std).multiply(2) + Average)
    lowerBand = ((Std).multiply(-2) + Average)
    bPercent = []
    for close, upper, lower in zip (newDF[valueLabel], upperBand, lowerBand):
        if upper - lower == 0:
            bPercent.append(0)
        else:
            bPercent.append((close - lower)/(upper - lower))
    newDF[series + 'bPercent'] = bPercent
    newDF = dateDF.merge(newDF, right_on = 'Date', left_on = 'Date', how = 'left')
    seriesDFList.append(newDF)
masterDF = seriesDFList[0].copy()
counter = 0
for df in seriesDFList:
    if counter != 0:
        masterDF= masterDF.merge(df, right_on = 'Date', left_on = 'Date')
    else:
        counter += 1
        


In [3]:
masterDF.fillna(method = 'ffill', inplace = True)
df = pd.read_csv('masterDF.csv', parse_dates = True)
df['Date'] = pd.to_datetime(df['Date'])
print(df)
print(masterDF)
df = df.merge(masterDF, right_on = 'Date', left_on = 'Date')
print(df)
df.to_csv('combinedDF.csv', index=False) 

             Date    Open    High      Low   Close    Volume  \
0      2010-02-09  19.935  20.063  19.6980  19.853   5457866   
1      2010-02-10  19.887  19.908  19.4950  19.744   3449712   
2      2010-02-11  19.744  19.987  19.4210  19.820   7398888   
3      2010-02-12  20.690  20.752  19.9610  20.239  12141187   
4      2010-02-16  20.414  20.479  20.2320  20.420   6950724   
...           ...     ...     ...      ...     ...       ...   
766200 2017-10-30  46.820  47.325  46.5521  46.800   2139479   
766201 2017-10-31  46.680  47.070  46.4400  46.460   1446143   
766202 2017-11-01  46.710  47.060  46.1600  46.220   1400055   
766203 2017-11-02  46.170  46.920  45.9900  46.850   1495963   
766204 2017-11-03  46.770  47.125  46.3700  46.810   1249319   

        VolumeZScoreTenDay  highVsLowPerc  dayPercentChange ticker  ...  \
0                -0.664861       0.018530         -0.004113      A  ...   
1                -1.696658       0.021185         -0.007191      A  ...   
2     

             Date     Open      High      Low    Close     Volume  \
0      2010-02-09   19.935   20.0630   19.698   19.853    5457866   
1      2010-02-09   40.491   41.0510   40.118   40.905    1046726   
2      2010-02-09   25.161   25.2940   24.938   25.125  176376562   
3      2010-02-09   24.718   24.9170   24.553   24.700    2383450   
4      2010-02-09   21.714   21.8520   21.593   21.596   27531462   
...           ...      ...       ...      ...      ...        ...   
766200 2017-11-03   62.160   66.0000   61.940   64.800    4671017   
766201 2017-11-03   29.510   29.5650   28.890   29.060    2916927   
766202 2017-11-03   78.550   80.2199   78.540   78.910    2583978   
766203 2017-11-03  110.080  111.0000  108.540  109.250    3918693   
766204 2017-11-03   46.770   47.1250   46.370   46.810    1249319   

        VolumeZScoreTenDay  highVsLowPerc  dayPercentChange ticker  ...  \
0                -0.664861       0.018530         -0.004113      A  ...   
1                -0.9